# Notebook notes

This notebook is for testing training

# Import libs

**setup autoreload**

In [1]:
%load_ext autoreload
%autoreload 2

**configure system path**

In [2]:
import os
import sys

# list directories where packages are stored
# note that the parent directory of te repo is added automatically
GYM_FOLDER = "gym-t99"

# get this notebook's current working directory
nb_cwd = os.getcwd()
# get name of its parent directory
nb_parent = os.path.dirname(nb_cwd)
# add packages to path
sys.path.insert(len(sys.path), nb_parent)
sys.path.insert(len(sys.path), os.path.join(nb_parent, GYM_FOLDER))

**import custom packages**

Use this code if you have not yet loaded gym_t99

In [3]:
import gym
registered = gym.envs.registration.registry.env_specs.copy()

import gym_t99
import t_net

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


**import custom packages**

Use this code if you have already loaded gym_t99, but now you introduced changes and want to re-load it again

In [7]:
import imp

# this code removes environment from gym's registry
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env not in registered:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]

imp.reload(gym_t99)

Remove basic-v0 from registry
Remove t99-v0 from registry


<module 'gym_t99' from '/Users/ianconceicao/239AS_T99_RL_Agent/gym-t99/gym_t99/__init__.py'>

**import out-of-the-box packages**

In [4]:
# import matplotlib
import matplotlib as plt
# configure matplotlib
%matplotlib inline

In [5]:
import numpy as np
from copy import deepcopy

In [6]:
# import torch
import torch
from torch import nn

# configure torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Workspace

In [7]:
# a simple random agent to play aginst and test
class RandomEnemy:
    def __init__(self):
        self.action_space = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    # this interface needs to be supported for any agent
    def action(self, state):
        return np.random.choice(self.action_space)

In [8]:
# a simple random agent to play aginst and test
class RandomEnemySC:    
    # this interface needs to be supported for any agent
    def action(self, observation):
        return np.random.choice(observation)

In [9]:
custom_gym = gym.make('gym_t99:t99sc-v0', num_players = 2, enemy=RandomEnemySC())

In [ ]:
#Randomly fill the boards, used to test graphics
from random import randrange
custom_gym = gym.make('gym_t99:t99-v0', num_players = 100, enemy=RandomEnemy())
for player in custom_gym.state.players:
    board = player.board
    if 0 == randrange(20):
        player.place = randrange(100)
    for i in range(board.shape[0]):
        for j in range(board.shape[1]):
            if board[i][j] == 10:
                continue
            odds = 10
            if i <= 20:
                odds = 20
            
            if randrange(odds) == 0:
                board[i][j] = randrange(8)

In [48]:
frame = custom_gym.render(mode="human")
# frame[0][-4, 2:6] = 1
# print(frame[0])

In [15]:
options, rewards = custom_gym._observe(0)

In [ ]:
action = {
    "reward": rewards[2],
    "state": options[2]
}

observation, _, done, _ = custom_gym.step(action)
options, rewards = observation

each member of "options" list is a State99 

what happens after call step?

1. state of the player controlled by Agent is updated. It becomes exactly as one passed in "aciton" dictionary
2. states of all other players are updated
3. after all of the updates happened, *.step() calculates all options for the NEXT step (for pieces it holds at hand and in swap). These options, along with rewards they will trigger, are returned.

In [22]:
custom_gym.state.players[0].incoming_garbage = [1, 1, 2, 2, 2, 3]

In [49]:
custom_gym.state.players[0].attack_strategy = 4

In [19]:
print(len(options))

68


In [21]:
# custom_gym.state.players[0].board[21, 5:12] = 1
print(options[2].players[0].board)
# print(custom_gym.state.players[0].piece_current.matrix)
# print(custom_gym.state.players[0].piece_current.x)
# print(custom_gym.state.players[0].piece_current.y)

[[10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  7  7  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  7  7  3  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  3  3  3  0  0

In [334]:
custom_gym._collision(custom_gym.state.players[0].board,
                      custom_gym.state.players[0].piece_current)

False

In [333]:
custom_gym.state.players[0].piece_current.y -= 1

In [159]:
custom_gym.step(1)

(None, None, None, {})

In [30]:
np.arange(10)

[autoreload of gym_t99 failed: Traceback (most recent call last):
  File "C:\Users\slavc\AppData\Roaming\Python\Python38\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\slavc\AppData\Roaming\Python\Python38\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\slavc\Desktop\UCLA\Courses\Spring_2021\239AS\239AS_T99_RL_Agent\venv_239AS\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\slavc\Desktop\UCLA\Courses\Spring_2021\239AS\239AS_T99_RL_Agent\venv_239AS\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\slavc\Desktop\UCLA\Courses\Spring_2021\239AS

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [29]:
a = np.array([True, False, False, False, True]).astype(bool)

In [32]:
x = np.where(a == True)[0]
print(x)
print(np.random.choice(x))
print(a)
print(np.sum(a))

[0 4]
4
[ True False False False  True]
2


In [22]:
board = custom_gym.state.players[0].board

In [31]:
temp = np.sum(board[:-3, 3:13], axis=1)
print(np.where(temp > 0)[0])

[17 18 19 20 21 22 23 24]
